In [1]:
from nltk import sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from tqdm.notebook import tqdm
from torchmetrics.text import ROUGEScore

In [2]:
df = pd.read_csv("../../../data/labelled/reviews/splits/test.csv")

In [3]:
rouge = ROUGEScore()

In [4]:
df.head(2)

,input_reviews,label_reviews
0,Below are the reviews of a product: \n- I purc...,pros:\n- Reduces glare\n- Very impressive\n- W...
1,Below are the reviews of a product: \n- Well w...,pros:\n- Huge capacity\n- Beautiful design\n- ...


In [5]:
vectorizer = TfidfVectorizer(stop_words=stopwords.words("english"))

In [6]:
preds = []
rouge_scores = {"rouge1":[], "rouge2":[], "rougeL": []}
for idx in tqdm(range(len(df))):
    sentences = sent_tokenize(df.input_reviews[idx])
    doc_term_matrix = vectorizer.fit_transform(sentences)
    scores = doc_term_matrix.sum(1).flatten().tolist()[0]
    sorted_pairs = sorted(zip(sentences,scores), key=lambda x: x[1], reverse=True)
    max_len = len(df.input_reviews[idx])//2
    cur_len, idx = 0, 0
    predicted_summary = ""
    while cur_len<=max_len:
        predicted_summary+=f"{sorted_pairs[idx][0]} "
        cur_len+=len(sorted_pairs[idx][0])
        idx+=1
    preds.append(predicted_summary.strip())
    rouge_all = rouge(predicted_summary, df.label_reviews[idx])
    rouge_scores['rouge1'].append(rouge_all['rouge1_fmeasure'].item())
    rouge_scores['rouge2'].append(rouge_all['rouge2_fmeasure'].item())
    rouge_scores['rougeL'].append(rouge_all['rougeL_fmeasure'].item())

  0%|          | 0/20000 [00:00<?, ?it/s]

In [7]:
df['rouge1'] = rouge_scores['rouge1']
df['rouge2'] = rouge_scores['rouge2']
df['rougeL'] = rouge_scores['rougeL']
df['tfidf'] = preds

In [8]:
df.to_csv("tfidf_reviews_scores.csv")

In [9]:
scores = {"rouge1": df.rouge1.mean(), "rouge2": df.rouge2.mean(), "rougeL": df.rougeL.mean()}

In [10]:
scores

{'rouge1': 0.14757152467968407,
 'rouge2': 0.010597576643963111,
 'rougeL': 0.07981185964783653}